In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Retrieve the dataset
categories = ["alt.atheism", "sci.med", "sci.electronics", "comp.graphics", "talk.politics.guns", "sci.crypt"]
data = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

In [ ]:
#retrieving the data in the right format for the analysis

x = data.data  # Text: just separation of the data information using the dot
y = data.target  # labels: just retrievinig the labels information using the dot


In [ ]:
print(x[1])



That wasn't my point. My point is that your implied position about the "Waco
Massacre" is an assertion cum political position. According to a CNN poll,
about 86% of Americans think the FBI acted properly, and a majority think
they should have done it (the tear gas) a lot sooner. A large majority also
believe the FBI was not at all responsible for the death of the victims,
Koresh was. (Let's avoid the kind of fruitless argument of the sort If A
hadn't X'd, then B might not have Y'd....)

What you call the "Big Brother Chip" is right now a proposal under
discussion, and it is not yet clear that strong crypto is going to be
banned.

Therefore neither justifies "any tactic is fair." Thus the statement is
nonsense. It is pernicious, because "any tactic" can lead to damaging
others.

As I see you know from the tone of your reply, there's nothing personal
intended.

David


In [ ]:
# vectorizing the data (converting words into numerical features)
vectorizer = TfidfVectorizer(stop_words='english', max_features = 30000)
x_tfid= vectorizer.fit_transform(x).toarray()
print(f"x_tfid shape: {x_tfid.shape}")


x_tfid shape: (5647, 30000)


In [ ]:
# Fn to remove documents with zero std (meaning no variability in the information)
def remove_zero_std_doc(x,y):
  #calculate the std across each instance/document
  std_dev = np.std(x, axis=1) #considering all the roxws (axis =1) compute the std in x

  #identify rows/documents with std equal to zero
  non_zero_std_indices = np.where(std_dev != 0)[0]

  #filter documents with std = 0
  x_filtered = x[non_zero_std_indices]
  y_filtered = y[non_zero_std_indices]

  return x_filtered, y_filtered


In [ ]:
#Applying the filtered functions to the data
x_filtered, y_filtered = remove_zero_std_doc(x_tfid, y)


print(x_filtered.shape)
print(y_filtered.shape)

(5489, 30000)
(5489,)


KMEANS

In [ ]:
# Init centroids
def init_centroids(data, k):
  """Select k initial centroids randomly from the data
  pARAM:dataset (n_samples, n_features); k; number of clusters
  return centroids: initial centroids
  """
  np.random.seed(42) #set random seed for reproducibility
  return data[np.random.choice(data.shape[0], k, replace=False)] #random selection of k datapoints as centroids


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# E-step: giving the centroids (u_k) compute the membership (pi_ik) for each datapoint
def e_step (data, centroids):
  #Distances computation with cosine similarity
  similarity = cosine_similarity(data,centroids)

  #Init a binary matrix with the same shape as similarity
  memberships = np.zeros_like(similarity)

  #assigning each datapoint to the closest cluster
  memberships[np.arange(similarity.shape[0]), np.argmax(similarity, axis=1)] =1

  return memberships

In [ ]:
# Objective function
def objective_function(data, centroids, memberships):
  #COMPUtation of the cosine distance since the J function minimizes distances, not maximizes similarities
  cosine_distances = 1 - cosine_similarity(data, centroids)

  #Multiply distances by the memberships matrix and sum
  j = np.sum(cosine_distances * memberships)

  return j

In [ ]:
# m-step: giving the membershios pi_ik, compute the new u_k updating them
def m_step(data,memberships):
  #I got the sum per cluster for each membership and reshape to be able to do the dot product
  cluster_sizes = np.sum(memberships, axis = 0).reshape(-1,1)

  #epsilon value to avoid zero divisions if some column has zero values
  cluster_sizes[cluster_sizes == 0] = 1e-10

  return np.dot(memberships.T, data) / cluster_sizes #memberships.T*data has the size (k,30000) representing the centroids of the K clusters in the space of 30000 characteres
  #each row of the resulting dot product matrix is a centroid

In [ ]:
#KMEANS
def kmeans(data, k, max_iter):
  #init
  centroids = init_centroids(data, k)

  #Storing the J's values after each iteration
  J_history = []

  for iteration in range(max_iter):

    #E-step
    memberships = e_step(data, centroids)

    #Objective function after e
    J_e = objective_function(data, centroids, memberships)
    J_history.append((iteration +1, 'E-step', J_e))
    print(f"Iteration {iteration + 1} (E-step), Objective Function J: {J_e}")

    #m-step: updating u_k
    centroids = m_step(data, memberships)

    # objective function after m
    J_after = objective_function(data, centroids, memberships)
    J_history.append((iteration + 1, 'M-step', J_after))
    print(f"Iteration {iteration + 1} (M-step), Objective Function J: {J_after}")

  # results
  return centroids, memberships, J_history

Calling K means

In [ ]:
# K means k=5
# Note: in iteration 20 there was no change in J so i set max_iter to 20
k = 5
max_iter = 20
centroids_5, memberships_5, J_history_5 = kmeans(x_filtered, k, max_iter)

Iteration 1 (E-step), Objective Function J: 5299.534496727587
Iteration 1 (M-step), Objective Function J: 4867.639585812889
Iteration 2 (E-step), Objective Function J: 4845.903171061871
Iteration 2 (M-step), Objective Function J: 4817.306306770699
Iteration 3 (E-step), Objective Function J: 4807.299121669654
Iteration 3 (M-step), Objective Function J: 4794.349126834223
Iteration 4 (E-step), Objective Function J: 4791.011173721903
Iteration 4 (M-step), Objective Function J: 4785.907367480945
Iteration 5 (E-step), Objective Function J: 4784.786692757064
Iteration 5 (M-step), Objective Function J: 4782.405044130991
Iteration 6 (E-step), Objective Function J: 4781.876319867414
Iteration 6 (M-step), Objective Function J: 4780.294221483591
Iteration 7 (E-step), Objective Function J: 4779.845596773439
Iteration 7 (M-step), Objective Function J: 4778.496980808092
Iteration 8 (E-step), Objective Function J: 4777.878140052192
Iteration 8 (M-step), Objective Function J: 4776.1287663074245
Iterati

In [ ]:
# K means k=10
k = 10
max_iter = 20
centroids_10, memberships_10, J_history_10 = kmeans(x_filtered, k, max_iter)

Iteration 1 (E-step), Objective Function J: 5241.976622464949
Iteration 1 (M-step), Objective Function J: 4800.020986802601
Iteration 2 (E-step), Objective Function J: 4767.203613175312
Iteration 2 (M-step), Objective Function J: 4719.911523979472
Iteration 3 (E-step), Objective Function J: 4705.631212880121
Iteration 3 (M-step), Objective Function J: 4684.856055494491
Iteration 4 (E-step), Objective Function J: 4680.3996163343645
Iteration 4 (M-step), Objective Function J: 4672.191947372377
Iteration 5 (E-step), Objective Function J: 4670.918520299231
Iteration 5 (M-step), Objective Function J: 4667.750103250934
Iteration 6 (E-step), Objective Function J: 4667.203949769289
Iteration 6 (M-step), Objective Function J: 4665.076770628042
Iteration 7 (E-step), Objective Function J: 4664.671001841251
Iteration 7 (M-step), Objective Function J: 4663.041480578778
Iteration 8 (E-step), Objective Function J: 4662.700554257115
Iteration 8 (M-step), Objective Function J: 4661.199573892047
Iterati

In [ ]:
# K means k=20
k = 20
max_iter = 20
centroids_20, memberships_20, J_history_20 = kmeans(x_filtered, k, max_iter)

Iteration 1 (E-step), Objective Function J: 5178.697957174853
Iteration 1 (M-step), Objective Function J: 4721.981066700662
Iteration 2 (E-step), Objective Function J: 4687.899981943985
Iteration 2 (M-step), Objective Function J: 4633.031120979629
Iteration 3 (E-step), Objective Function J: 4615.370660845388
Iteration 3 (M-step), Objective Function J: 4585.483208858279
Iteration 4 (E-step), Objective Function J: 4578.576434969599
Iteration 4 (M-step), Objective Function J: 4562.245032222249
Iteration 5 (E-step), Objective Function J: 4559.282118497912
Iteration 5 (M-step), Objective Function J: 4550.254621892907
Iteration 6 (E-step), Objective Function J: 4548.966577301223
Iteration 6 (M-step), Objective Function J: 4544.105561598023
Iteration 7 (E-step), Objective Function J: 4543.403544922439
Iteration 7 (M-step), Objective Function J: 4540.702470921031
Iteration 8 (E-step), Objective Function J: 4540.484644326094
Iteration 8 (M-step), Objective Function J: 4538.762964342882
Iteratio

In [ ]:
#retrieving the clusters witk k=10
# getting from the membership matrix of 5000 instances x 10 columns (one per cluster) the max value
# from each row (axis=1) that denotes the membership to the cluster (the membership matrix have 0 or 1)
# this retrieves a matrix with the shape of the instances (aprox 5000) with one column indicating the belonging to each cluster
clusters = np.argmax(memberships_10, axis=1)

In [ ]:
print(clusters.shape)

(5489,)


Gini and Purity index

In [ ]:
# Function for Purity
# Purity measures how well the clusters align with the true labels
from collections import Counter

def purity(clusters, labels):
    correct_labels = 0  # Initialize a counter to keep track of correctly assigned points
    n_clusters = len(np.unique(clusters))  # Total number of clusters

    for cluster in range(n_clusters):
        indices = np.where(clusters == cluster)[0]  # Get the indices of points in this cluster
        cluster_labels = labels[indices]  # Get the true labels for the points in this cluster
        most_freq = Counter(cluster_labels).most_common(1)[0][1]  # Find the most frequent label in the cluster
        correct_labels += most_freq  # Add the count of correct predictions for this cluster

    return correct_labels / len(labels)  # Compute and return purity as the fraction of correct predictions

In [ ]:
ng_purity = purity(clusters, y_filtered)
print(f"Purity: {ng_purity}")

Purity: 0.7023137183457825


In [ ]:
# Function for Gini Index
# The Gini index measures the heterogeneity of labels within clusters

def gini_index(clusters, Y):
    n_samples = len(Y)  # Total number of samples in the dataset
    n_clusters = len(np.unique(clusters))  # Total number of unique clusters
    total_gini = 0  # Initialize the weighted Gini index accumulator

    for cluster in range(n_clusters):
        # Get indices of data points assigned to the current cluster
        indices = np.where(clusters == cluster)[0]
        cluster_labels = Y[indices]  # Get the true labels of the points in this cluster

        # Compute the total number of points in the cluster
        total_points = len(cluster_labels)
        if total_points == 0:
            continue  # Skip the cluster if it's empty

        # Count the occurrences of each label in the cluster
        label_counts = Counter(cluster_labels)
        # Compute the proportions of each label
        proportions = np.array([count / total_points for count in label_counts.values()])

        # Calculate the Gini index for this cluster
        gini = 1 - np.sum(proportions**2)  # Gini index formula: 1 - sum(p_j^2)
        total_gini += total_points * gini  # Weight the Gini index by cluster size

    # Calculate the global Gini index by averaging over all clusters
    global_gini = total_gini / n_samples
    return global_gini

In [ ]:
#gini computation
gini_ng = gini_index(clusters, y_filtered)
print(f"Gini: {gini_ng}")

Gini: 0.4245116807605418
